<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)   
* [RLlib Section](#7)            

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [1]:
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

import warnings
warnings.filterwarnings('ignore')

import yfinance as yf

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [3]:
# from config.py TRAIN_START_DATE is a string
config.TRAIN_START_DATE

'2017-11-09'

In [4]:
# from config.py TRAIN_END_DATE is a string
config.TRAIN_END_DATE

'2021-04-30'

In [5]:
CRYPTO_TICKER = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD']
#CRYPTO_TICKER = ['ETH-USD', 'BTC-USD']
GLOBAL_TICKER = ['^STOXX50E', '^GSPC']

TOTAL_TICKER = CRYPTO_TICKER + GLOBAL_TICKER

In [6]:
df = YahooDownloader(start_date = '2017-11-09',
                     end_date = '2022-04-30',
                     ticker_list = CRYPTO_TICKER).fetch_data()

# We start gather data on the 2017-11-09 since it is the first day where we have data on eth on yfinance

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (6533, 8)


In [7]:
print(CRYPTO_TICKER)

['BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD']


In [8]:
df.shape

(6533, 8)

In [9]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2017-11-08,7141.379883,7776.419922,7114.020020,7459.689941,4602200064,BTC-USD,2
1,2017-11-09,2.053140,2.174230,1.893940,1.990770,19192200,BNB-USD,3
2,2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,3226249984,BTC-USD,3
3,2017-11-09,308.644989,329.451996,307.056000,320.884003,893249984,ETH-USD,3
4,2017-11-09,1.010870,1.013270,0.996515,1.008180,358188000,USDT-USD,3


In [10]:
df[df['tic'] == 'ETH-USD'].sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,893249984,ETH-USD,3
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,885985984,ETH-USD,4
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,842300992,ETH-USD,5
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,1613479936,ETH-USD,6
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,1041889984,ETH-USD,0


In [11]:
df[df['tic'] == 'BTC-USD'].sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2017-11-08,7141.379883,7776.419922,7114.020020,7459.689941,4602200064,BTC-USD,2
1,2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,3226249984,BTC-USD,3
2,2017-11-10,7173.729980,7312.000000,6436.870117,6618.140137,5208249856,BTC-USD,4
3,2017-11-11,6618.609863,6873.149902,6204.220215,6357.600098,4908680192,BTC-USD,5
4,2017-11-12,6295.450195,6625.049805,5519.009766,5950.069824,8957349888,BTC-USD,6


In [12]:
df['tic'].iloc[0]

'BTC-USD'

In [13]:
def get_ticker_indicator_df(df, ticker_to_trade, ticker_indicator, 
                            stock_indicators = ['open', 'high', 'low', 'close', 'volume']):
    
    df_indicator = df[df['tic'] == ticker_indicator].sort_values(['date','tic'],ignore_index=True)
    for indicator in stock_indicators:
        df_indicator = df_indicator.rename(columns = {indicator : f'{indicator}_{df_indicator.tic.iloc[0]}'})
    df_indicator = df_indicator.drop(['tic', 'day'], axis = 1)
    
    df = df[df['tic'] == ticker_to_trade]
    df = pd.merge(df, df_indicator, on = 'date')
    return df

In [14]:
def add_ticker_indicators_df(df, ticker_to_trade, tickers_indicators,
                            stock_indicators = ['open', 'high', 'low', 'close', 'volume']):
    
    dfs_to_merge = []
    for ticker_indicator in tickers_indicators:
        if(ticker_indicator != ticker_to_trade):
            dfs_to_merge.append(get_ticker_indicator_df(df, ticker_to_trade, ticker_indicator))

    df_indicators = dfs_to_merge[0]
    for counter in range(1, len(dfs_to_merge)):
        df_indicators = pd.merge(df_indicators, dfs_to_merge[counter], on = ['date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'day'])
    return df_indicators


In [15]:
df_indicators = add_ticker_indicators_df(df, 'ETH-USD', CRYPTO_TICKER)

In [16]:
df_indicators

,date,open,high,low,close,volume,tic,day,open_BTC-USD,high_BTC-USD,...,open_USDT-USD,high_USDT-USD,low_USDT-USD,close_USDT-USD,volume_USDT-USD,open_BNB-USD,high_BNB-USD,low_BNB-USD,close_BNB-USD,volume_BNB-USD
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,893249984,ETH-USD,3,7446.830078,7446.830078,...,1.010870,1.013270,0.996515,1.008180,358188000,2.053140,2.174230,1.893940,1.990770,19192200
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,885985984,ETH-USD,4,7173.729980,7312.000000,...,1.006500,1.024230,0.995486,1.006010,756446016,2.007730,2.069470,1.644780,1.796840,11155000
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,842300992,ETH-USD,5,6618.609863,6873.149902,...,1.005980,1.026210,0.995799,1.008990,746227968,1.786280,1.917750,1.614290,1.670470,8178150
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,1613479936,ETH-USD,6,6295.450195,6625.049805,...,1.006020,1.105910,0.967601,1.012470,1466060032,1.668890,1.672800,1.462560,1.519690,15298700
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,1041889984,ETH-USD,0,5938.250000,6811.189941,...,1.004480,1.029290,0.975103,1.009350,767884032,1.526010,1.735020,1.517600,1.686620,12238800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,2022-04-25,2922.990234,3018.415527,2804.507080,3009.393555,22332690614,ETH-USD,0,39472.605469,40491.753906,...,1.000346,1.000393,1.000103,1.000145,73256647805,399.129913,404.873901,383.190674,404.350281,1910707893
1629,2022-04-26,3008.946289,3026.415039,2786.253174,2808.298340,19052045399,ETH-USD,1,40448.421875,40713.890625,...,1.000131,1.000265,1.000040,1.000073,69068297656,404.268860,407.084503,382.080994,385.483063,1671963898
1630,2022-04-27,2808.645996,2911.877441,2802.273438,2888.929688,17419284041,ETH-USD,2,38120.300781,39397.917969,...,1.000059,1.000284,1.000013,1.000153,61043327313,385.562164,394.458527,384.077698,391.445831,1512587369
1631,2022-04-28,2888.849854,2973.135010,2861.821533,2936.940918,18443524633,ETH-USD,3,39241.429688,40269.464844,...,1.000155,1.000292,1.000028,1.000191,68392086439,391.438660,408.232452,388.877655,406.718201,2116381172


In [17]:
for ticker in CRYPTO_TICKER:
    print(f'{ticker} : {len(df[df.tic == ticker])}')

BTC-USD : 1634
ETH-USD : 1633
USDT-USD : 1633
BNB-USD : 1633


We have one more value for BTC (it starts one day before) we will delete it.
For the global ticker we don't have the values on the week-end days so we will interpolate these values.
There is many ways to interpolate these values, we will start by taking the values of Friday for Saturday and the values of Monday for Sunday

In [186]:
# For now, we only start with crypto data so we don't have to do the interpolation
# We just drop the first row of the df

#df = df.drop([0])

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

## STILL NEED TO ADD FE ON THE NEW TICKERS

In [18]:
config.STOCK_INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [19]:
# First we redefine 'config.INDICATORS' with the indicators from the others currency
# We store 'config.INDICATORS' on 'config.STOCK_INDICATORS'


stock_indicators = ['date,', 'tic', 'open', 'high', 'low', 'close', 'volume', 'vix', 'turbulence']

for indicator in df_indicators.columns:
    if((indicator not in stock_indicators) and (indicator not in config.INDICATORS)):
        config.INDICATORS.append(indicator)
        
config.INDICATORS.remove('date')
config.INDICATORS.remove('day')

config.INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma',
 'open_BTC-USD',
 'high_BTC-USD',
 'low_BTC-USD',
 'close_BTC-USD',
 'volume_BTC-USD',
 'open_USDT-USD',
 'high_USDT-USD',
 'low_USDT-USD',
 'close_USDT-USD',
 'volume_USDT-USD',
 'open_BNB-USD',
 'high_BNB-USD',
 'low_BNB-USD',
 'close_BNB-USD',
 'volume_BNB-USD']

In [20]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

#processed = fe.preprocess_data(df)
processed = fe.preprocess_data(df_indicators)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1125, 8)
Successfully added vix
Successfully added turbulence index


In [21]:
processed

,date,open,high,low,close,volume,tic,day,open_BTC-USD,high_BTC-USD,...,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,893249984,ETH-USD,3,7446.830078,7446.830078,...,0.000000,340.659367,279.477626,0.000000,-66.666667,100.000000,320.884003,320.884003,10.500000,0.000000
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,885985984,ETH-USD,4,7173.729980,7312.000000,...,-0.485311,340.659367,279.477626,0.000000,-66.666667,100.000000,310.068497,310.068497,11.290000,0.000000
2,2017-11-13,307.024994,328.415009,307.024994,316.716003,1041889984,ETH-USD,0,5938.250000,6811.189941,...,0.085548,328.844885,294.931910,47.098193,70.924692,5.038490,311.888397,311.888397,11.500000,0.000000
3,2017-11-14,316.763000,340.177002,316.763000,337.631012,1069680000,ETH-USD,1,6561.479980,6764.979980,...,1.164355,342.098056,290.259610,63.231222,149.767947,36.129409,316.178833,316.178833,11.590000,0.000000
4,2017-11-15,337.963989,340.911987,329.812988,333.356995,722665984,ETH-USD,2,6634.759766,7342.250000,...,1.642529,345.622870,291.642842,59.401690,125.701937,37.758889,318.632856,318.632856,13.130000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119,2022-04-22,2986.938721,3024.854492,2942.358643,2964.835693,16782795477,ETH-USD,4,40525.863281,40777.757812,...,-28.661926,3489.221440,2807.256441,47.787023,-103.041884,18.061940,3197.143449,2965.672990,28.209999,0.030387
1120,2022-04-25,2922.990234,3018.415527,2804.507080,3009.393555,22332690614,ETH-USD,0,39472.605469,40491.753906,...,-41.412147,3269.975413,2867.939773,48.979120,-99.024734,31.438966,3180.887712,2983.048226,27.020000,0.045202
1121,2022-04-26,3008.946289,3026.415039,2786.253174,2808.298340,19052045399,ETH-USD,1,40448.421875,40713.890625,...,-55.429932,3276.840980,2824.734850,44.745521,-117.456030,33.012639,3164.778402,2983.777604,33.520000,1.415885
1122,2022-04-27,2808.645996,2911.877441,2802.273438,2888.929688,17419284041,ETH-USD,2,38120.300781,39397.917969,...,-59.348717,3253.471575,2813.669758,46.657987,-112.067341,33.012639,3149.854907,2985.574569,31.600000,0.211302


In [22]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

# Change 'date' to index of the df

#processed_full = processed_full.set_index('date')

In [24]:
processed_full.sort_values(['date','tic'],ignore_index=False).head()

,date,tic,open,high,low,close,volume,day,open_BTC-USD,high_BTC-USD,...,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2017-11-09,ETH-USD,308.644989,329.451996,307.056000,320.884003,8.932500e+08,3.0,7446.830078,7446.830078,...,0.000000,340.659367,279.477626,0.000000,-66.666667,100.000000,320.884003,320.884003,10.50,0.0
1,2017-11-10,ETH-USD,320.670990,324.717987,294.541992,299.252991,8.859860e+08,4.0,7173.729980,7312.000000,...,-0.485311,340.659367,279.477626,0.000000,-66.666667,100.000000,310.068497,310.068497,11.29,0.0
4,2017-11-13,ETH-USD,307.024994,328.415009,307.024994,316.716003,1.041890e+09,0.0,5938.250000,6811.189941,...,0.085548,328.844885,294.931910,47.098193,70.924692,5.038490,311.888397,311.888397,11.50,0.0
5,2017-11-14,ETH-USD,316.763000,340.177002,316.763000,337.631012,1.069680e+09,1.0,6561.479980,6764.979980,...,1.164355,342.098056,290.259610,63.231222,149.767947,36.129409,316.178833,316.178833,11.59,0.0
6,2017-11-15,ETH-USD,337.963989,340.911987,329.812988,333.356995,7.226660e+08,2.0,6634.759766,7342.250000,...,1.642529,345.622870,291.642842,59.401690,125.701937,37.758889,318.632856,318.632856,13.13,0.0


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

To apply this environment to the trade only on the Ethereum crypto, we will allow to trade only on 'ETH-USD

In [192]:
##### CHANGES ADDED TO STOCKTRADINGENV :
##### - change cost percetage of trades


import gym
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gym import spaces
from gym.utils import seeding
from stable_baselines3.common.vec_env import DummyVecEnv
from typing import List
matplotlib.use("Agg")

# from stable_baselines3.common.logger import Logger, KVWriter, CSVOutputFormat


class StockTradingEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""

    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        df: pd.DataFrame,
        stock_dim: int,
        hmax: int,
        initial_amount: int,
        num_stock_shares: List[int],
        buy_cost_pct: List[float],
        sell_cost_pct: List[float],
        reward_scaling: float,
        state_space: int,
        action_space: int,
        tech_indicator_list: List[str],
        turbulence_threshold=None,
        risk_indicator_col="turbulence",
        make_plots: bool =False,
        print_verbosity=10,
        day=0,
        initial=True,
        previous_state=[],
        model_name="",
        mode="",
        iteration="",
    ):
        self.day = day
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.num_stock_shares=num_stock_shares
        self.initial_amount = initial_amount # get the initial cash
        self.buy_cost_pct = buy_cost_pct
        self.sell_cost_pct = sell_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.action_space,))
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.state_space,)
        )
        self.data = self.df.loc[self.day, :]
        self.terminal = False
        self.make_plots = make_plots
        self.print_verbosity = print_verbosity
        self.turbulence_threshold = turbulence_threshold
        self.risk_indicator_col = risk_indicator_col
        self.initial = initial
        self.previous_state = previous_state
        self.model_name = model_name
        self.mode = mode
        self.iteration = iteration
        # initalize state
        self.state = self._initiate_state()

        # initialize reward
        self.reward = 0
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        self.episode = 0
        # memorize all the total balance change
        self.asset_memory = [self.initial_amount+np.sum(np.array(self.num_stock_shares)*np.array(self.state[1:1+self.stock_dim]))] # the initial total asset is calculated by cash + sum (num_share_stock_i * price_stock_i)
        self.rewards_memory = []
        self.actions_memory = []
        self.state_memory=[] # we need sometimes to preserve the state in the middle of trading process 
        self.date_memory = [self._get_date()]
        #         self.logger = Logger('results',[CSVOutputFormat])
        # self.reset()
        self._seed()

    def _sell_stock(self, index, action):
        def _do_sell_normal():
            if self.state[index + 2*self.stock_dim + 1]!=True : # check if the stock is able to sell, for simlicity we just add it in techical index
            # if self.state[index + 1] > 0: # if we use price<0 to denote a stock is unable to trade in that day, the total asset calculation may be wrong for the price is unreasonable
                # Sell only if the price is > 0 (no missing data in this particular date)
                # perform sell action based on the sign of the action
                if self.state[index + self.stock_dim + 1] > 0:
                    # Sell only if current asset is > 0
                    sell_num_shares = min(
                        abs(action), self.state[index + self.stock_dim + 1]
                    )
                    sell_amount = (
                        self.state[index + 1]
                        * sell_num_shares
                        * (1 - self.sell_cost_pct[index])
                    )
                    # update balance
                    self.state[0] += sell_amount

                    self.state[index + self.stock_dim + 1] -= sell_num_shares
                    self.cost += (
                        self.state[index + 1] * sell_num_shares * self.sell_cost_pct[index]
                    )
                    self.trades += 1
                else:
                    sell_num_shares = 0
            else:
                sell_num_shares = 0

            return sell_num_shares

        # perform sell action based on the sign of the action
        if self.turbulence_threshold is not None:
            if self.turbulence >= self.turbulence_threshold:
                if self.state[index + 1] > 0:
                    # Sell only if the price is > 0 (no missing data in this particular date)
                    # if turbulence goes over threshold, just clear out all positions
                    if self.state[index + self.stock_dim + 1] > 0:
                        # Sell only if current asset is > 0
                        sell_num_shares = self.state[index + self.stock_dim + 1]
                        sell_amount = (
                            self.state[index + 1]
                            * sell_num_shares
                            * (1 - self.sell_cost_pct[index])
                        )
                        # update balance
                        self.state[0] += sell_amount
                        self.state[index + self.stock_dim + 1] = 0
                        self.cost += (
                            self.state[index + 1] * sell_num_shares * self.sell_cost_pct
                        )
                        self.trades += 1
                    else:
                        sell_num_shares = 0
                else:
                    sell_num_shares = 0
            else:
                sell_num_shares = _do_sell_normal()
        else:
            sell_num_shares = _do_sell_normal()

        return sell_num_shares

    def _buy_stock(self, index, action):
        def _do_buy():
            if self.state[index + 2*self.stock_dim+ 1] !=True: # check if the stock is able to buy
            # if self.state[index + 1] >0:
                # Buy only if the price is > 0 (no missing data in this particular date)
                available_amount = self.state[0] / (self.state[index + 1]*(1 + self.buy_cost_pct[index])) # when buying stocks, we should consider the cost of trading when calculating available_amount, or we may be have cash<0
                # print('available_amount:{}'.format(available_amount))

                # update balance
                buy_num_shares = min(available_amount, action)
                buy_amount = (
                    self.state[index + 1] * buy_num_shares * (1 + self.buy_cost_pct[index])
                )
                self.state[0] -= buy_amount

                self.state[index + self.stock_dim + 1] += buy_num_shares

                self.cost += self.state[index + 1] * buy_num_shares * self.buy_cost_pct[index]
                self.trades += 1
            else:
                buy_num_shares = 0

            return buy_num_shares

        # perform buy action based on the sign of the action
        if self.turbulence_threshold is None:
            buy_num_shares = _do_buy()
        else:
            if self.turbulence < self.turbulence_threshold:
                buy_num_shares = _do_buy()
            else:
                buy_num_shares = 0
                pass

        return buy_num_shares

    def _make_plot(self):
        plt.plot(self.asset_memory, "r")
        plt.savefig("results/account_value_trade_{}.png".format(self.episode))
        plt.close()

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1
        if self.terminal:
            # print(f"Episode: {self.episode}")
            if self.make_plots:
                self._make_plot()
            end_total_asset = self.state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
            )
            df_total_value = pd.DataFrame(self.asset_memory)
            tot_reward = (
                self.state[0]
                + sum(
                    np.array(self.state[1 : (self.stock_dim + 1)])
                    * np.array(
                        self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)]
                    )
                )
                - self.asset_memory[0]
            ) # initial_amount is only cash part of our initial asset
            df_total_value.columns = ["account_value"]
            df_total_value["date"] = self.date_memory
            df_total_value["daily_return"] = df_total_value["account_value"].pct_change(
                1
            )
            if df_total_value["daily_return"].std() != 0:
                sharpe = (
                    (252 ** 0.5)
                    * df_total_value["daily_return"].mean()
                    / df_total_value["daily_return"].std()
                )
            df_rewards = pd.DataFrame(self.rewards_memory)
            df_rewards.columns = ["account_rewards"]
            df_rewards["date"] = self.date_memory[:-1]
            if self.episode % self.print_verbosity == 0:
                print(f"day: {self.day}, episode: {self.episode}")
                print(f"begin_total_asset: {self.asset_memory[0]:0.2f}")
                print(f"end_total_asset: {end_total_asset:0.2f}")
                print(f"total_reward: {tot_reward:0.2f}")
                print(f"total_cost: {self.cost:0.2f}")
                print(f"total_trades: {self.trades}")
                if df_total_value["daily_return"].std() != 0:
                    print(f"Sharpe: {sharpe:0.3f}")
                print("=================================")

            if (self.model_name != "") and (self.mode != ""):
                df_actions = self.save_action_memory()
                df_actions.to_csv(
                    "results/actions_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    )
                )
                df_total_value.to_csv(
                    "results/account_value_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    ),
                    index=False,
                )
                df_rewards.to_csv(
                    "results/account_rewards_{}_{}_{}.csv".format(
                        self.mode, self.model_name, self.iteration
                    ),
                    index=False,
                )
                plt.plot(self.asset_memory, "r")
                plt.savefig(
                    "results/account_value_{}_{}_{}.png".format(
                        self.mode, self.model_name, self.iteration
                    ),
                    index=False,
                )
                plt.close()

            # Add outputs to logger interface
            # logger.record("environment/portfolio_value", end_total_asset)
            # logger.record("environment/total_reward", tot_reward)
            # logger.record("environment/total_reward_pct", (tot_reward / (end_total_asset - tot_reward)) * 100)
            # logger.record("environment/total_cost", self.cost)
            # logger.record("environment/total_trades", self.trades)

            return self.state, self.reward, self.terminal, {}

        else:
            actions = actions * self.hmax  # actions initially is scaled between 0 to 1
            actions = actions.astype(
                int
            )  # convert into integer because we can't by fraction of shares
            if self.turbulence_threshold is not None:
                if self.turbulence >= self.turbulence_threshold:
                    actions = np.array([-self.hmax] * self.stock_dim)
            begin_total_asset = self.state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
            )
            # print("begin_total_asset:{}".format(begin_total_asset))

            argsort_actions = np.argsort(actions)
            sell_index = argsort_actions[: np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][: np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
                # print(f"Num shares before: {self.state[index+self.stock_dim+1]}")
                # print(f'take sell action before : {actions[index]}')
                actions[index] = self._sell_stock(index, actions[index]) * (-1)
                # print(f'take sell action after : {actions[index]}')
                # print(f"Num shares after: {self.state[index+self.stock_dim+1]}")

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                actions[index] = self._buy_stock(index, actions[index])

            self.actions_memory.append(actions)

            # state: s -> s+1
            self.day += 1
            self.data = self.df.loc[self.day, :]
            if self.turbulence_threshold is not None:
                if len(self.df.tic.unique()) == 1:
                    self.turbulence = self.data[self.risk_indicator_col]
                elif len(self.df.tic.unique()) > 1:
                    self.turbulence = self.data[self.risk_indicator_col].values[0]
            self.state = self._update_state()

            end_total_asset = self.state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
            )
            self.asset_memory.append(end_total_asset)
            self.date_memory.append(self._get_date())
            self.reward = end_total_asset - begin_total_asset
            self.rewards_memory.append(self.reward)
            self.reward = self.reward * self.reward_scaling
            self.state_memory.append(self.state) # add current state in state_recorder for each step

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        # initiate state
        self.state = self._initiate_state()

        if self.initial:
            self.asset_memory = [self.initial_amount+np.sum(np.array(self.num_stock_shares)*np.array(self.state[1:1+self.stock_dim]))]
        else:
            previous_total_asset = self.previous_state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(
                    self.previous_state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)]
                )
            )
            self.asset_memory = [previous_total_asset]

        self.day = 0
        self.data = self.df.loc[self.day, :]
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        self.terminal = False
        # self.iteration=self.iteration
        self.rewards_memory = []
        self.actions_memory = []
        self.date_memory = [self._get_date()]

        self.episode += 1

        return self.state

    def render(self, mode="human", close=False):
        return self.state

    def _initiate_state(self):
        if self.initial:
            # For Initial State
            if len(self.df.tic.unique()) > 1:
                # for multiple stock
                state = (
                    [self.initial_amount]
                    + self.data.close.values.tolist()
                    + self.num_stock_shares
                    + sum(
                        [
                            self.data[tech].values.tolist()
                            for tech in self.tech_indicator_list
                        ],
                        [],
                    )
                ) # append initial stocks_share to initial state, instead of all zero 
            else:
                # for single stock
                state = (
                    [self.initial_amount]
                    + [self.data.close]
                    + [0] * self.stock_dim
                    + sum([[self.data[tech]] for tech in self.tech_indicator_list], [])
                )
        else:
            # Using Previous State
            if len(self.df.tic.unique()) > 1:
                # for multiple stock
                state = (
                    [self.previous_state[0]]
                    + self.data.close.values.tolist()
                    + self.previous_state[
                        (self.stock_dim + 1) : (self.stock_dim * 2 + 1)
                    ]
                    + sum(
                        [
                            self.data[tech].values.tolist()
                            for tech in self.tech_indicator_list
                        ],
                        [],
                    )
                )
            else:
                # for single stock
                state = (
                    [self.previous_state[0]]
                    + [self.data.close]
                    + self.previous_state[
                        (self.stock_dim + 1) : (self.stock_dim * 2 + 1)
                    ]
                    + sum([[self.data[tech]] for tech in self.tech_indicator_list], [])
                )
        return state

    def _update_state(self):
        if len(self.df.tic.unique()) > 1:
            # for multiple stock
            state = (
                [self.state[0]]
                + self.data.close.values.tolist()
                + list(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
                + sum(
                    [
                        self.data[tech].values.tolist()
                        for tech in self.tech_indicator_list
                    ],
                    [],
                )
            )

        else:
            # for single stock
            state = (
                [self.state[0]]
                + [self.data.close]
                + list(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
                + sum([[self.data[tech]] for tech in self.tech_indicator_list], [])
            )

        return state

    def _get_date(self):
        if len(self.df.tic.unique()) > 1:
            date = self.data.date.unique()[0]
        else:
            date = self.data.date
        return date

    # add save_state_memory to preserve state in the trading process 
    def save_state_memory(self):
        if len(self.df.tic.unique()) > 1:
            # date and close price length must match actions length
            date_list = self.date_memory[:-1]
            df_date = pd.DataFrame(date_list)
            df_date.columns = ["date"]

            state_list = self.state_memory
            df_states = pd.DataFrame(state_list,columns=['cash','Bitcoin_price','Gold_price','Bitcoin_num','Gold_num','Bitcoin_Disable','Gold_Disable'])
            df_states.index = df_date.date
            # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        else:
            date_list = self.date_memory[:-1]
            state_list = self.state_memory
            df_states = pd.DataFrame({"date": date_list, "states": state_list})
        # print(df_states)
        return df_states

    def save_asset_memory(self):
        date_list = self.date_memory
        asset_list = self.asset_memory
        # print(len(date_list))
        # print(len(asset_list))
        df_account_value = pd.DataFrame(
            {"date": date_list, "account_value": asset_list}
        )
        return df_account_value

    def save_action_memory(self):
        if len(self.df.tic.unique()) > 1:
            # date and close price length must match actions length
            date_list = self.date_memory[:-1]
            df_date = pd.DataFrame(date_list)
            df_date.columns = ["date"]

            action_list = self.actions_memory
            df_actions = pd.DataFrame(action_list)
            df_actions.columns = self.data.tic.values
            df_actions.index = df_date.date
            # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        else:
            date_list = self.date_memory[:-1]
            action_list = self.actions_memory
            df_actions = pd.DataFrame({"date": date_list, "actions": action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs


## Training data split: 2017-11-09 to 2021-04-30
## Trade data split: 2021-04-30 to 2022-04-30

In [69]:
train = data_split(processed_full, '2021-04-1','2022-01-31')
trade = data_split(processed_full, '2022-02-01','2022-04-30')
print(len(train))
print(len(trade))

204
61


In [73]:
train.tail()

,date,tic,open,high,low,close,volume,day,open_BTC-USD,high_BTC-USD,...,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
199,2022-01-24,ETH-USD,2535.891113,2537.208496,2172.301270,2440.352295,2.822080e+10,0.0,36275.734375,37247.519531,...,-307.967206,3760.063394,2415.608652,34.033923,-185.228346,64.075308,3328.073494,3729.954915,29.900000,0.683425
200,2022-01-25,ETH-USD,2440.393555,2498.507080,2359.384766,2455.935059,1.617978e+10,1.0,36654.804688,37444.570312,...,-317.951843,3724.706553,2341.520303,34.333823,-163.433238,64.075308,3274.360392,3703.705351,31.160000,0.001121
201,2022-01-26,ETH-USD,2455.579102,2705.784180,2417.683105,2468.030273,2.122991e+10,2.0,36950.515625,38825.410156,...,-321.186322,3696.063018,2275.126044,34.572662,-135.718954,44.318202,3222.043148,3676.557320,31.959999,0.000118
202,2022-01-27,ETH-USD,2467.188477,2510.451660,2328.452148,2423.001221,1.612652e+10,3.0,36841.878906,37148.324219,...,-323.652276,3692.792033,2201.376107,34.095069,-143.772526,47.241674,3176.113420,3645.366447,30.490000,0.139504
203,2022-01-28,ETH-USD,2421.646973,2548.778564,2367.833740,2547.092041,1.466623e+10,4.0,37128.445312,37952.878906,...,-311.996963,3682.872481,2156.807597,36.592185,-120.149281,43.775717,3140.065430,3613.732898,27.660000,0.592117


In [75]:
trade

,date,tic,open,high,low,close,volume,day,open_BTC-USD,high_BTC-USD,...,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2022-02-01,ETH-USD,2687.898926,2802.315430,2682.621826,2792.117188,1.319485e+10,1.0,38481.765625,39115.132812,...,-225.797544,3524.362466,2098.290415,41.443892,-43.470546,21.018567,2996.238778,3492.574060,21.959999,0.310297
1,2022-02-02,ETH-USD,2791.958984,2802.212158,2630.120361,2682.854004,1.387630e+10,2.0,38743.714844,38834.617188,...,-208.925528,3467.403072,2098.706351,39.984433,-48.255245,23.901414,2960.287899,3468.628503,22.090000,0.509817
2,2022-02-03,ETH-USD,2682.226074,2712.482666,2587.783447,2679.162598,1.275551e+10,3.0,36944.804688,37154.601562,...,-193.620265,3390.229085,2112.796452,39.935282,-51.788706,26.222712,2923.124764,3443.309167,24.350000,0.007192
3,2022-02-04,ETH-USD,2681.057617,2983.586914,2675.443848,2983.586914,1.898722e+10,4.0,37149.265625,41527.785156,...,-155.137903,3323.613329,2144.717824,45.636368,-4.449415,4.419658,2904.231429,3420.389994,23.219999,3.282749
4,2022-02-07,ETH-USD,3057.422119,3182.528320,3002.927002,3142.470703,1.519706e+10,0.0,42406.781250,44401.863281,...,-56.338891,3209.766714,2207.298740,48.404260,50.633010,9.011830,2887.931551,3359.395988,22.860001,0.655323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2022-04-22,ETH-USD,2986.938721,3024.854492,2942.358643,2964.835693,1.678280e+10,4.0,40525.863281,40777.757812,...,-28.661926,3489.221440,2807.256441,47.787023,-103.041884,18.061940,3197.143449,2965.672990,28.209999,0.030387
57,2022-04-25,ETH-USD,2922.990234,3018.415527,2804.507080,3009.393555,2.233269e+10,0.0,39472.605469,40491.753906,...,-41.412147,3269.975413,2867.939773,48.979120,-99.024734,31.438966,3180.887712,2983.048226,27.020000,0.045202
58,2022-04-26,ETH-USD,3008.946289,3026.415039,2786.253174,2808.298340,1.905205e+10,1.0,40448.421875,40713.890625,...,-55.429932,3276.840980,2824.734850,44.745521,-117.456030,33.012639,3164.778402,2983.777604,33.520000,1.415885
59,2022-04-27,ETH-USD,2808.645996,2911.877441,2802.273438,2888.929688,1.741928e+10,2.0,38120.300781,39397.917969,...,-59.348717,3253.471575,2813.669758,46.657987,-112.067341,33.012639,3149.854907,2985.574569,31.600000,0.211302


In [28]:
config.INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma',
 'open_BTC-USD',
 'high_BTC-USD',
 'low_BTC-USD',
 'close_BTC-USD',
 'volume_BTC-USD',
 'open_USDT-USD',
 'high_USDT-USD',
 'low_USDT-USD',
 'close_USDT-USD',
 'volume_USDT-USD',
 'open_BNB-USD',
 'high_BNB-USD',
 'low_BNB-USD',
 'close_BNB-USD',
 'volume_BNB-USD']

In [76]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 1, State Space: 26


In [77]:
# CARE, HERE I MODIFIED 0.001 to 0 to annualate trade fees for our model to trade more

buy_cost_list = sell_cost_list = [0] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [78]:
e_train_gym.action_space

Box([-1.], [1.], (1,), float32)

## Environment for Training



In [79]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [80]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [81]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [82]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 449       |
|    iterations         | 100       |
|    time_elapsed       | 1         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.42     |
|    explained_variance | -0.0196   |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 4.72      |
|    reward             | 3.0515537 |
|    std                | 1         |
|    value_loss         | 23.2      |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 457       |
|    iterations         | 200       |
|    time_elapsed       | 2         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -1.42     |
|    explained_variance | 0.147     |
|    learning_rate      | 0.0007    |
|    n_updat

KeyboardInterrupt: 

### Model 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

### Model 3: PPO

In [95]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [96]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 203, episode: 350
begin_total_asset: 1000000.00
end_total_asset: 1637368.40
total_reward: 637368.40
total_cost: 0.00
total_trades: 198
Sharpe: 1.292
----------------------------------
| time/              |           |
|    fps             | 596       |
|    iterations      | 1         |
|    time_elapsed    | 3         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 1.2798537 |
----------------------------------
day: 203, episode: 360
begin_total_asset: 1000000.00
end_total_asset: 1229299.85
total_reward: 229299.85
total_cost: 0.00
total_trades: 185
Sharpe: 0.731
------------------------------------------
| time/                   |              |
|    fps                  | 576          |
|    iterations           | 2            |
|    time_elapsed         | 7            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0017786048 |
|    clip_fraction        | 0.004

day: 203, episode: 440
begin_total_asset: 1000000.00
end_total_asset: 1073290.96
total_reward: 73290.96
total_cost: 0.00
total_trades: 176
Sharpe: 0.422
-----------------------------------------
| time/                   |             |
|    fps                  | 582         |
|    iterations           | 10          |
|    time_elapsed         | 35          |
|    total_timesteps      | 20480       |
| train/                  |             |
|    approx_kl            | 0.007163315 |
|    clip_fraction        | 0.0341      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.41       |
|    explained_variance   | 0.0908      |
|    learning_rate        | 0.00025     |
|    loss                 | 52.2        |
|    n_updates            | 90          |
|    policy_gradient_loss | -0.00401    |
|    reward               | 6.868934    |
|    std                  | 0.988       |
|    value_loss           | 102         |
-----------------------------------------
day: 20

day: 203, episode: 520
begin_total_asset: 1000000.00
end_total_asset: 1478594.21
total_reward: 478594.21
total_cost: 0.00
total_trades: 168
Sharpe: 1.240
------------------------------------------
| time/                   |              |
|    fps                  | 588          |
|    iterations           | 18           |
|    time_elapsed         | 62           |
|    total_timesteps      | 36864        |
| train/                  |              |
|    approx_kl            | 0.0046887575 |
|    clip_fraction        | 0.021        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.37        |
|    explained_variance   | 0.0914       |
|    learning_rate        | 0.00025      |
|    loss                 | 114          |
|    n_updates            | 170          |
|    policy_gradient_loss | -0.00308     |
|    reward               | 0.45959383   |
|    std                  | 0.954        |
|    value_loss           | 232          |
-----------------------------

### Model 4: TD3

In [85]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [86]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

day: 203, episode: 200
begin_total_asset: 1000000.00
end_total_asset: 1039731.29
total_reward: 39731.29
total_cost: 0.00
total_trades: 203
Sharpe: 0.549
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 78       |
|    time_elapsed    | 10       |
|    total_timesteps | 816      |
| train/             |          |
|    actor_loss      | 9.99e+08 |
|    critic_loss     | 1.68e+17 |
|    learning_rate   | 0.001    |
|    n_updates       | 612      |
|    reward          | 5.065428 |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 66       |
|    time_elapsed    | 24       |
|    total_timesteps | 1632     |
| train/             |          |
|    actor_loss      | 9.19e+08 |
|    critic_loss     | 1.48e+17 |
|    learning_rate   | 0.001    |
|    n_updates       | 1428     |
|    reward          | 5.065428

---------------------------------
| time/              |          |
|    episodes        | 68       |
|    fps             | 58       |
|    time_elapsed    | 238      |
|    total_timesteps | 13872    |
| train/             |          |
|    actor_loss      | 3.35e+08 |
|    critic_loss     | 1.58e+16 |
|    learning_rate   | 0.001    |
|    n_updates       | 13668    |
|    reward          | 5.065428 |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 72       |
|    fps             | 58       |
|    time_elapsed    | 252      |
|    total_timesteps | 14688    |
| train/             |          |
|    actor_loss      | 3.09e+08 |
|    critic_loss     | 1.39e+16 |
|    learning_rate   | 0.001    |
|    n_updates       | 14484    |
|    reward          | 5.065428 |
---------------------------------
day: 203, episode: 270
begin_total_asset: 1000000.00
end_total_asset: 1039731.29
total_reward: 39731.29
total_cost: 

---------------------------------
| time/              |          |
|    episodes        | 132      |
|    fps             | 56       |
|    time_elapsed    | 473      |
|    total_timesteps | 26928    |
| train/             |          |
|    actor_loss      | 1.44e+08 |
|    critic_loss     | 2.43e+15 |
|    learning_rate   | 0.001    |
|    n_updates       | 26724    |
|    reward          | 5.065428 |
---------------------------------
day: 203, episode: 330
begin_total_asset: 1000000.00
end_total_asset: 1039731.29
total_reward: 39731.29
total_cost: 0.00
total_trades: 203
Sharpe: 0.549
---------------------------------
| time/              |          |
|    episodes        | 136      |
|    fps             | 56       |
|    time_elapsed    | 488      |
|    total_timesteps | 27744    |
| train/             |          |
|    actor_loss      | 1.41e+08 |
|    critic_loss     | 2.3e+15  |
|    learning_rate   | 0.001    |
|    n_updates       | 27540    |
|    reward          | 5.065428

### Model 5: SAC

In [83]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 1000000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [84]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=60000)

day: 203, episode: 20
begin_total_asset: 1000000.00
end_total_asset: 1283057.93
total_reward: 283057.93
total_cost: 0.00
total_trades: 87
Sharpe: 0.965
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 54       |
|    time_elapsed    | 15       |
|    total_timesteps | 816      |
| train/             |          |
|    actor_loss      | 5.55e+08 |
|    critic_loss     | 2.3e+16  |
|    ent_coef        | 0.107    |
|    ent_coef_loss   | 22.1     |
|    learning_rate   | 0.0001   |
|    n_updates       | 715      |
|    reward          | 0.0      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 51       |
|    time_elapsed    | 31       |
|    total_timesteps | 1632     |
| train/             |          |
|    actor_loss      | 5.43e+08 |
|    critic_loss     | 1.56e+16 |
|    ent_coef        | 0.117    

day: 203, episode: 80
begin_total_asset: 1000000.00
end_total_asset: 1000000.00
total_reward: 0.00
total_cost: 0.00
total_trades: 0
---------------------------------
| time/              |          |
|    episodes        | 64       |
|    fps             | 48       |
|    time_elapsed    | 269      |
|    total_timesteps | 13056    |
| train/             |          |
|    actor_loss      | 1.81e+08 |
|    critic_loss     | 2.5e+15  |
|    ent_coef        | 0.365    |
|    ent_coef_loss   | 9.89     |
|    learning_rate   | 0.0001   |
|    n_updates       | 12955    |
|    reward          | 0.0      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 68       |
|    fps             | 48       |
|    time_elapsed    | 286      |
|    total_timesteps | 13872    |
| train/             |          |
|    actor_loss      | 1.66e+08 |
|    critic_loss     | 2.63e+15 |
|    ent_coef        | 0.396    |
|    ent_coef_loss

day: 203, episode: 140
begin_total_asset: 1000000.00
end_total_asset: 1000000.00
total_reward: 0.00
total_cost: 0.00
total_trades: 0
---------------------------------
| time/              |          |
|    episodes        | 124      |
|    fps             | 48       |
|    time_elapsed    | 526      |
|    total_timesteps | 25296    |
| train/             |          |
|    actor_loss      | 9.25e+07 |
|    critic_loss     | 1.09e+15 |
|    ent_coef        | 1.24     |
|    ent_coef_loss   | -2.14    |
|    learning_rate   | 0.0001   |
|    n_updates       | 25195    |
|    reward          | 0.0      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 128      |
|    fps             | 48       |
|    time_elapsed    | 543      |
|    total_timesteps | 26112    |
| train/             |          |
|    actor_loss      | 8.94e+07 |
|    critic_loss     | 3.99e+14 |
|    ent_coef        | 1.35     |
|    ent_coef_los

KeyboardInterrupt: 

## Trading
Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [57]:
data_risk_indicator = processed_full[(processed_full.date<'2021-04-30') & (processed_full.date>='2017-11-09')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [58]:
insample_risk_indicator.vix.describe()

count    872.000000
mean      20.167087
std        9.645215
min        9.150000
25%       13.530000
50%       17.460000
75%       23.450001
max       82.690002
Name: vix, dtype: float64

In [59]:
insample_risk_indicator.vix.quantile(0.996)

73.79052062988268

In [60]:
insample_risk_indicator.turbulence.describe()

count    872.000000
mean       0.783612
std        3.473756
min        0.000000
25%        0.000000
50%        0.040732
75%        0.425810
max       79.096379
Name: turbulence, dtype: float64

In [61]:
insample_risk_indicator.turbulence.quantile(0.996)

12.651610411007068

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [62]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [63]:
trade.head()

,date,tic,open,high,low,close,volume,day,open_BTC-USD,high_BTC-USD,...,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2021-04-30,ETH-USD,2757.734131,2796.054932,2728.169922,2773.207031,2.977718e+10,4.0,53568.664062,57900.718750,...,174.464045,2809.551665,2009.865010,65.531558,178.777421,37.231803,2297.672754,2020.136731,18.610001,0.009635
1,2021-05-03,ETH-USD,2951.175781,3450.037842,2951.175781,3431.086182,4.917429e+10,0.0,56620.273438,58973.308594,...,255.668805,3172.448543,1920.251848,73.820994,251.430755,61.774865,2403.676428,2098.781047,18.309999,13.716108
2,2021-05-04,ETH-USD,3431.131592,3523.585938,3180.742676,3253.629395,6.240205e+10,1.0,57214.179688,57214.179688,...,275.333805,3285.599999,1888.952833,69.101079,225.053368,63.546820,2442.359981,2127.453619,19.480000,1.072426
3,2021-05-05,ETH-USD,3240.554688,3541.462646,3213.101562,3522.783203,4.833420e+10,2.0,53252.164062,57911.363281,...,309.074107,3450.061169,1824.858363,71.918229,214.699672,63.980650,2489.523181,2158.587646,19.150000,1.251543
4,2021-05-06,ETH-USD,3524.930908,3598.895996,3386.239990,3490.880371,4.430039e+10,3.0,57441.308594,58363.316406,...,329.441650,3580.888165,1799.924750,71.123115,195.560187,65.395020,2535.273230,2188.049756,18.389999,0.125952


In [97]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


In [98]:
df_account_value.shape

(252, 2)

In [99]:
df_account_value.tail()

,date,account_value
247,2022-04-22,1.926699e+06
248,2022-04-25,1.928659e+06
249,2022-04-26,1.905734e+06
250,2022-04-27,1.922103e+06
251,2022-04-28,1.936122e+06


In [100]:
df_actions.head(10)

,date,actions
0,2021-04-30,[39]
1,2021-05-03,[-39]
2,2021-05-04,[0]
3,2021-05-05,[0]
4,2021-05-06,[0]
5,2021-05-07,[0]
6,2021-05-10,[0]
7,2021-05-11,[0]
8,2021-05-12,[0]
9,2021-05-13,[0]


In [101]:
for i in range(len(df_actions['actions'])):
    print(df_actions['actions'].iloc[i])

[39]
[-39]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[22]
[-22]
[22]
[-22]
[0]
[0]
[0]
[70]
[-70]
[0]
[0]
[0]
[24]
[89]
[78]
[78]
[10]
[-32]
[-32]
[10]
[-32]
[45]
[78]
[78]
[78]
[7]
[0]
[0]
[0]
[0]
[-32]
[-29]
[55]
[0]
[0]
[0]
[0]
[0]
[0]
[-31]
[31]
[0]
[0]
[0]
[-93]
[-31]
[78]
[34]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[-93]
[-93]
[-44]
[-93]
[-93]
[-47]
[22]
[22]
[-44]
[0]
[0]
[0]
[0]
[0]
[0]
[22]
[22]
[22]
[-66]
[0]
[0]
[0]
[0]
[89]
[22]
[-93]
[-18]
[0]
[89]
[22]
[70]
[22]
[-44]
[22]
[22]
[22]
[89]
[22]
[22]
[88]
[-93]
[-93]
[78]
[78]
[32]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[-93]
[-93]
[-93]
[-93]
[-76]
[0]
[0]
[0]
[89]
[22]
[-93]
[-18]
[0]
[22]
[22]
[22]
[-66]
[22]
[-22]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[70]
[22]
[22]
[22]
[22]
[22]
[70]
[-93]
[-93]
[-64]
[0]
[22]
[70]
[10]
[78]
[-32]
[22]
[-93]
[-26]
[-31]
[-20]
[39]
[-32]
[-7]
[0]
[0]
[78]
[89]
[22]
[-93]
[-93]
[-3]
[0]
[22]
[22]
[-44]
[0]
[89]
[-44]
[22]
[-67]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[

In [94]:
trade['open']

0     2687.898926
1     2791.958984
2     2682.226074
3     2681.057617
4     3057.422119
         ...     
56    2986.938721
57    2922.990234
58    3008.946289
59    2808.645996
60    2888.849854
Name: open, Length: 61, dtype: float64

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [102]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.936122
Cumulative returns     0.936122
Annual volatility      0.399532
Sharpe ratio           1.857564
Calmar ratio           4.065697
Stability              0.776096
Max drawdown          -0.230249
Omega ratio            1.617255
Sortino ratio          3.274655
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.660898
Daily value at risk   -0.047391
dtype: float64


In [103]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="ETH-USD", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ETH-USD: No data found for this date range, symbol may be delisted


AttributeError: Can only use .dt accessor with datetimelike values

In [151]:
df_account_value.loc[0,'date']

'2021-04-30'

In [152]:
df_account_value.loc[len(df_account_value)-1,'date']

'2022-04-28'

<a id='6.2'></a>
## 7.2 BackTestPlot

In [51]:
print("==============Compare to ETH===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = 'ETH-USD', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

==============Compare to ETH===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (364, 8)


Start date,2021-04-30
End date,2022-04-28
Total months,12
,Backtest
Annual return,-7.4%
Cumulative returns,-7.4%
Annual volatility,89.7%
Sharpe ratio,0.37
Calmar ratio,-0.13
Stability,0.06
Max drawdown,-57.1%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'